<a href="https://colab.research.google.com/github/Hemanthvadapalli/LeapTask/blob/main/8Puzzle_Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

# Goal state of the 8-puzzle problem
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

In [ ]:
# Function to flatten the 2D puzzle board into a 1D list
def flatten(state):
    return [tile for row in state for tile in row]

# Function to calculate fitness score based on the number of misplaced tiles
def fitness(state):
    flat_state = flatten(state)
    flat_goal = flatten(goal_state)
    return sum([1 if flat_state[i] != flat_goal[i] else 0 for i in range(9)])

In [ ]:
# Function to find the blank tile's (0) position
def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

In [ ]:
# Function to generate neighbors by moving the blank tile in possible directions
def get_neighbors(state):
    x, y = find_blank(state)
    neighbors = []
    moves = []

    # Possible moves: Up, Down, Left, Right
    move_list = [(-1, 0, 'Up'), (1, 0, 'Down'), (0, -1, 'Left'), (0, 1, 'Right')]

    for dx, dy, move in move_list:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < 3 and 0 <= new_y < 3:
            neighbor = [row[:] for row in state]  # Copy the state
            # Swap the blank tile with the adjacent tile
            neighbor[x][y], neighbor[new_x][new_y] = neighbor[new_x][new_y], neighbor[x][y]
            neighbors.append(neighbor)
            moves.append(move)

    return neighbors, moves

In [ ]:
# Function to generate random initial states for the population
def generate_initial_population(initial_state, size):
    population = [initial_state]
    for _ in range(size - 1):
        individual = random.sample(range(9), 9)
        population.append([individual[:3], individual[3:6], individual[6:]])
    return population


In [ ]:
# Function to select two parents for crossover based on their fitness
def select_parents(population):
    fitness_scores = [(individual, fitness(individual)) for individual in population]
    fitness_scores.sort(key=lambda x: x[1])
    return fitness_scores[0][0], fitness_scores[1][0]

In [ ]:
# Function to perform crossover between two parents and generate offspring
def crossover(parent1, parent2):
    flat_parent1 = flatten(parent1)
    flat_parent2 = flatten(parent2)

    # Perform single-point crossover
    crossover_point = random.randint(0, 8)

    child1 = flat_parent1[:crossover_point] + flat_parent2[crossover_point:]
    child2 = flat_parent2[:crossover_point] + flat_parent1[crossover_point:]

    # Ensure that children are valid puzzle states
    child1 = make_valid_state(child1)
    child2 = make_valid_state(child2)

    return [child1[:3], child1[3:6], child1[6:]], [child2[:3], child2[3:6], child2[6:]]

In [ ]:
# Function to make sure the crossover result is a valid state
def make_valid_state(flat_child):
    missing = set(range(9)) - set(flat_child)
    for i in range(len(flat_child)):
        if flat_child.count(flat_child[i]) > 1:
            flat_child[i] = missing.pop()
    return flat_child

In [ ]:
# Function to mutate a child by swapping two random tiles
def mutate(child, mutation_rate=0.1):
    if random.random() < mutation_rate:
        flat_child = flatten(child)
        idx1, idx2 = random.sample(range(9), 2)
        flat_child[idx1], flat_child[idx2] = flat_child[idx2], flat_child[idx1]
        return [flat_child[:3], flat_child[3:6], flat_child[6:]]
    return child

In [ ]:
# Genetic Algorithm for solving the 8-Puzzle Problem with move tracking
def genetic_algorithm(initial_state, population_size=100, generations=1000, mutation_rate=0.1):
    population = generate_initial_population(initial_state, population_size)
    best_moves = []

    for generation in range(generations):
        parent1, parent2 = select_parents(population)

        # If a solution is found (fitness = 0)
        if fitness(parent1) == 0:
            return parent1, best_moves, generation

        # Generate new population through crossover and mutation
        new_population = []
        while len(new_population) < population_size:
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            # Track neighbors and moves for children
            neighbors1, moves1 = get_neighbors(child1)
            neighbors2, moves2 = get_neighbors(child2)

            # Select one random valid move for each child
            chosen_index1 = random.choice(range(len(neighbors1)))
            chosen_index2 = random.choice(range(len(neighbors2)))

            child1 = neighbors1[chosen_index1]
            child2 = neighbors2[chosen_index2]

            # Append moves to the move history
            best_moves.append(moves1[chosen_index1])
            best_moves.append(moves2[chosen_index2])

            new_population.append(child1)
            new_population.append(child2)

        population = new_population

    # Return the best solution, the moves made, and the number of generations
    best_solution = select_parents(population)[0]
    return best_solution, best_moves, generations


In [ ]:
# Helper function to print the puzzle state
def print_puzzle(state):
    for row in state:
        print(row)
    print()


In [ ]:
# Running the Genetic Algorithm with user input
initial_state = [
    [1, 2, 3],
    [4, 5, 0],
    [7, 8, 6]
]

In [ ]:
population_size = 100
generations = 1000

solution, moves, gen = genetic_algorithm(initial_state, population_size=population_size, generations=generations)

print(f"Solution found in {gen} generations:")
print_puzzle(solution)

Solution found in 1 generations:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [ ]:
print("Moves made to reach the solution (by 0 tile):")
print(moves)

Moves made to reach the solution (by 0 tile):
['Up', 'Down', 'Down', 'Left', 'Down', 'Down', 'Down', 'Down', 'Down', 'Left', 'Up', 'Down', 'Down', 'Left', 'Left', 'Up', 'Down', 'Up', 'Down', 'Up', 'Up', 'Down', 'Right', 'Down', 'Right', 'Down', 'Down', 'Up', 'Right', 'Up', 'Right', 'Left', 'Up', 'Right', 'Left', 'Up', 'Down', 'Left', 'Right', 'Up', 'Right', 'Left', 'Down', 'Down', 'Right', 'Left', 'Right', 'Left', 'Left', 'Right', 'Right', 'Up', 'Down', 'Down', 'Right', 'Left', 'Down', 'Left', 'Right', 'Up', 'Up', 'Right', 'Right', 'Down', 'Left', 'Up', 'Right', 'Up', 'Right', 'Down', 'Down', 'Up', 'Down', 'Left', 'Down', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Right', 'Right', 'Left', 'Up', 'Down', 'Up', 'Down', 'Down', 'Down', 'Down', 'Right', 'Right', 'Left', 'Left', 'Down', 'Right', 'Down', 'Down', 'Left']


In [ ]:
# Printing the number of Steps

In [ ]:
print(len(moves))

100
